In [1]:
import pyarrow.parquet as pq
trips = pq.read_table('/Users/admin/softeer/week5/yellow_tripdata_2024-05.parquet')
trips = trips.to_pandas()

In [3]:
import pyarrow.parquet as pq
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, to_date

# PyArrow를 사용하여 Parquet 파일 읽기
trips = pq.read_table('/Users/admin/softeer/week5/yellow_tripdata_2024-05.parquet')

# Spark Session 생성
spark = SparkSession.builder \
    .appName("NYC_TLC_Data_Analysis") \
    .master("local[*]") \
    .getOrCreate()

# PyArrow 테이블을 Spark DataFrame으로 변환
taxi_trip_df = spark.createDataFrame(trips.to_pandas())

# 데이터 클리닝
taxi_trip_df = taxi_trip_df.filter((col('fare_amount') > 0) & (col('trip_distance') > 0))

# 필요한 열 선택 및 데이터 변환
taxi_trip_df = taxi_trip_df.select('pickup_datetime', 'trip_distance', 'fare_amount')
taxi_trip_df = taxi_trip_df.withColumn('pickup_date', to_date(col('pickup_datetime')))

# 집계 로직: 일별 집계
daily_metrics_df = taxi_trip_df.groupBy('pickup_date') \
    .agg(
        _sum('fare_amount').alias('total_revenue'),
        _sum('trip_distance').alias('total_trip_distance'),
        count('*').alias('total_trips')
    )

# 전체 집계 결과 계산
total_trips = taxi_trip_df.count()
total_revenue = taxi_trip_df.agg(_sum('fare_amount').alias('total_revenue')).collect()[0]['total_revenue']
avg_trip_distance = taxi_trip_df.agg(avg('trip_distance').alias('avg_trip_distance')).collect()[0]['avg_trip_distance']

# 전체 집계 결과 출력
print(f"Total Trips: {total_trips}")
print(f"Total Revenue: {total_revenue}")
print(f"Average Trip Distance: {avg_trip_distance}")

# 일별 집계 결과 출력
daily_metrics_df.show()

# 결과 저장 (로컬 파일 시스템에 저장)
output_path = "/Users/admin/softeer/week5/output"
daily_metrics_df.write.csv(f"{output_path}/daily_metrics", header=True)

# Spark 세션 종료
spark.stop()


Py4JJavaError: An error occurred while calling o32.applyModifiableSettings.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:92)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:75)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.Spark